In [ ]:
import nibiru
import nibiru.common # Common values and types to run
import nibiru.msg # Messages package for transactions
import json
from datetime import datetime, timedelta
import time

In [ ]:
network = nibiru.Network.customnet()

In [ ]:
VALIDATOR_MNEMONIC="guard cream sadness conduct invite crumble clock pudding hole grit liar hotel maid produce squeeze return argue turtle know drive eight casino maze host"

In [ ]:
tx_config = nibiru.TxConfig(tx_type=nibiru.common.TxType.BLOCK)

validator = (
    nibiru.Sdk
        .authorize(VALIDATOR_MNEMONIC) # This allows us to recover the wallet with a mnemonic
        .with_network(network, insecure=True)
        .with_config(tx_config)
)

# Arbitrage

Let's look at the current btc:usd mark price in our chain

In [ ]:
reserves = validator.query.vpool.reserve_assets(pair="ubtc:unusd")

In [ ]:
reserves

We can get the price by dividing the quote asset reserve by the base asset reserve:

In [ ]:
reserves["quote_asset_reserve"] / reserves["base_asset_reserve"]

BTC:USD spot is currently at 19,759.40USD. Which means that we need short positions opened to move the mark price down to the index price

In [ ]:
import math
def quote_needed_to_move_price(current_price, target_price, quote_reserve) -> float:
    """
    Compute how much quote is needed to move the pool current_price to target_price. This does not include fees.
    Demonstration is left as an exercise (involve playing a bit with the xy=k formula)

    Returns:
        float: Amount of quote to swap
    """
    qp = target_price / current_price
    return -(quote_reserve / math.sqrt(qp) - quote_reserve)

In [ ]:
quote_needed_to_move_price(20000, 19759.40, reserves["quote_asset_reserve"])

In [ ]:
# This might fail is you are trying to move the price too much in a single block.
# There's a fluctuation limit as to how much you can move the price by yourself, which is usually
# around 10% (query the vpool to check how much it is)

tx_output = validator.tx.execute_msgs(
    nibiru.msg.MsgOpenPosition(
        sender=validator.address, 
        token_pair="ubtc:unusd", 
        side=nibiru.common.Side.SELL, 
        quote_asset_amount=6069820187248.625, 
        leverage=1, 
        base_asset_amount_limit=100000000000)
)

In [ ]:
reserves = validator.query.vpool.reserve_assets(pair="ubtc:unusd")
reserves["quote_asset_reserve"] / reserves["base_asset_reserve"]